# Getting Started with OpenHands SDK

This notebook is an introduction to building agents with the [OpenHands SDK](https://docs.openhands.dev/sdk).

**What you'll learn:**
1. How to create an LLM and Agent
2. How to use built-in tools (Terminal, FileEditor, TaskTracker)
3. How to add a custom tool (Tavily web search with citations)
4. **Built-in persistence** - auto-save & resume with 2 parameters

**Why OpenHands SDK?**
- **Persistence built-in** - No graph setup, no external DB, just 2 params
- Production-ready with type safety (Pydantic)
- Built-in tools for coding tasks
- Works with any LLM (OpenAI, Anthropic, etc.)
- State-of-the-art performance on coding benchmarks

## Step 1: Install

In [1]:
!pip install openhands-sdk python-dotenv tavily-python -q

## Step 2: Load Environment

Create a `.env` file:
```
LLM_API_KEY=sk-your-openai-key
LLM_MODEL=openai/gpt-5
TAVILY_API_KEY=tvly-your-tavily-key
```

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-5')}")

Model: openai/gpt-5


## Step 3: Create the LLM

In [3]:
from openhands.sdk import LLM

llm = LLM(
    model="openai/gpt-5",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

print(f"✓ LLM ready: {llm.model}")

✓ LLM ready: openai/gpt-5


## Step 4: Define Tavily Search Tool

In [4]:
from typing import List
from pydantic import Field
from tavily import TavilyClient
from openhands.sdk import Action, Observation, ToolDefinition, TextContent
from openhands.sdk.tool import register_tool, ToolExecutor

tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

# OpenHands SDK requires 4 components for custom tools (vs just a function in other frameworks):

# 1. ACTION: Defines the INPUT schema for the tool
#    - This becomes the JSON schema the LLM sees for function calling
#    - Pydantic validates inputs before your code runs
class SearchAction(Action):
    query: str = Field(description="Search query")

# 2. OBSERVATION: Defines the OUTPUT schema for the tool
#    - to_llm_content() formats the result for the LLM to read
#    - Enables structured outputs and type safety
class SearchObservation(Observation):
    results: str = Field(description="Results")
    @property
    def to_llm_content(self): return [TextContent(text=self.results)]

# 3. EXECUTOR: The actual implementation (this is where your logic goes)
#    - Receives validated Action, must return Observation
#    - Gets conversation context if needed
class SearchExecutor(ToolExecutor):
    def __call__(self, action: SearchAction, conversation=None) -> SearchObservation:
        r = tavily.search(query=action.query, max_results=5)
        # Include title, snippet, AND URL for citations
        text = "\n\n".join([
            f"**{x['title']}**\n{x['content'][:300]}\nSource: {x['url']}"
            for x in r['results']
        ])
        return SearchObservation(results=text or "No results")

# 4. TOOL DEFINITION: Packages everything together for registration
#    - Links Action + Observation + Executor
#    - create() factory allows access to conversation state if needed
class SearchTool(ToolDefinition[SearchAction, SearchObservation]):
    @classmethod
    def create(cls, conv_state) -> List["SearchTool"]:
        return [cls(description="Web search via Tavily", action_type=SearchAction,
                    observation_type=SearchObservation, executor=SearchExecutor())]

register_tool("TavilySearch", SearchTool.create)
print("✓ Tavily tool ready")

✓ Tavily tool ready


## Step 5: Create the Agent with Tools (including Tavily)

In [5]:
from openhands.sdk import Agent, Tool
from openhands.tools.terminal import TerminalTool
from openhands.tools.file_editor import FileEditorTool
from openhands.tools.task_tracker import TaskTrackerTool

agent = Agent(
    llm=llm,
    tools=[
        Tool(name=TerminalTool.name),
        Tool(name=FileEditorTool.name),
        Tool(name=TaskTrackerTool.name),
        Tool(name="TavilySearch"),  # Our custom Tavily tool
    ],
)

print("✓ Agent ready with tools:", [t.name for t in agent.tools])

✓ Agent ready with tools: ['terminal', 'file_editor', 'task_tracker', 'TavilySearch']


## Step 6: Create a Conversation with Persistence

OpenHands SDK has **built-in persistence** - just add 2 parameters:
- `persistence_dir`: Where to save (auto-creates directory)
- `conversation_id`: Unique session ID (use same ID to resume)

**Every event is auto-saved.** If interrupted, re-run with same ID to resume from exact point.

In [ ]:
from openhands.sdk import Conversation

cwd = os.getcwd()

# Built-in persistence: just 2 parameters!
# Compare to LangGraph which requires: StateGraph, nodes, edges, checkpointer class
conversation = Conversation(
    agent=agent,
    workspace=cwd,
    persistence_dir="./.conversations",     # Where to save state
    conversation_id="simple-agent-demo",    # Session ID (change for new session)
)

print(f"✓ Conversation ready in: {cwd}")
print(f"✓ Persistence: ./.conversations/simple-agent-demo/")

[01/23/26 18:58:38] INFO     Created new conversation 4def395d-6488-4d69-a1da-2ebf2e2ac266             ]8;id=278753;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=549386;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('4def395d-6488-4d69-a1da-2ebf2e2ac266'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir': None, 'max_iterations': 500,                    
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-5', 'api_key':                                   
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 272000,                         
                             'max_output_tokens': 128000, 'stream': False, 'drop_params': True,                    
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'terminal', 'params': {}},              
                             {'name': 'file_editor', 'params': {}}, {'name': 'task_tracker', 'params':             
                             {}}, {'name': 'TavilySearch', 'params': {}}], 'mcp_config': {},                       
                             'include_default_tools': ['FinishTool', 'ThinkTool'],                                 
                             'system_prompt_filename': 'system_prompt.j2', 'security_policy_filename':             
                             'security_policy.j2', 'system_prompt_kwargs': {'llm_security_analyzer':               
                             True}, 'kind': 'Agent'}                                                               

[01/23/26 18:58:38] INFO     TaskTrackerExecutor initialized with save_dir: None                  ]8;id=917375;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/task_tracker/definition.py\definition.py]8;;\:]8;id=73590;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/task_tracker/definition.py#161\161]8;;\

[01/23/26 18:58:38] INFO     Auto-detected: Using SubprocessTerminal (tmux not available)            ]8;id=510895;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/factory.py\factory.py]8;;\:]8;id=964171;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/factory.py#120\120]8;;\

[01/23/26 18:58:38] INFO     FileEditor initialized with cwd:                                          ]8;id=167038;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py\editor.py]8;;\:]8;id=563363;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py#95\95]8;;\
                             /Users/rajivshah/Code/openhands-deep-agent                                            

[01/23/26 18:58:38] INFO     Using shell: /bin/bash                                      ]8;id=751910;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/subprocess_terminal.py\subprocess_terminal.py]8;;\:]8;id=710826;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/subprocess_terminal.py#106\106]8;;\

[01/23/26 18:58:39] INFO     TerminalExecutor initialized with working_dir:                              ]8;id=953613;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/impl.py\impl.py]8;;\:]8;id=467998;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/impl.py#59\59]8;;\
                             /Users/rajivshah/Code/openhands-deep-agent, username: None, terminal_type:            
                             TerminalSession                                                                       

[01/23/26 18:58:39] INFO     Loaded 4 tools from spec: ['terminal', 'file_editor', 'task_tracker',      ]8;id=388607;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py\base.py]8;;\:]8;id=494784;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py#281\281]8;;\
                             'search']                                                                             

System Prompt ─────────────────────────────────────────────────────────────────────────────────────────────────────

System Prompt:
You are OpenHands agent, a helpful AI assistant that can interact with a computer to solve tasks.

<ROLE>
* Your primary role is to assist users by executing commands, modifying code, and solving technical problems 
effectively. You should be thorough, methodical, and prioritize quality over speed.
* If the user asks a question, like "why is X happening", don't try to fix the problem. Just give an answer to the 
question.
</ROLE>

<MEMORY>
* Use `AGENTS.md` under the repository root as your persistent memory for repository-specific knowledge and 
context.
* Add important insights, patterns, and learnings to this file to improve future task performance.
* This repository skill is automatically loaded for every conversation and helps maintain context across sessions.
* For more information about skills, see: https://docs.openhands.dev/overview/skills
</MEMORY>

<EFFICIENCY>
* Each action you take is somewhat expensive. Wherever possible, combine multiple actions into a single action, 
e.g. combine multiple bash commands into one, using sed and grep to edit/view multiple files at once.
* When exploring the codebase, use efficient tools like find, grep, and git commands with appropriate filters to 
minimize unnecessary operations.
</EFFICIENCY>

<FILE_SYSTEM_GUIDELINES>
* When a user provides a file path, do NOT assume it's relative to the current working directory. First explore the
file system to locate the file before working on it.
* If asked to edit a file, edit the file directly, rather than creating a new file with a different filename.
* For global search-and-replace operations, consider using `sed` instead of opening file editors multiple times.
* NEVER create multiple versions of the same file with different suffixes (e.g., file_test.py, file_fix.py, 
file_simple.py). Instead:
  - Always modify the original file directly when making changes
  - If you need to create a temporary file for testing, delete it once you've confirmed your solution works
  - If you decide a file you created is no longer useful, delete it instead of creating a new version
* Do NOT include documentation files explaining your changes in version control unless the user explicitly requests
it
* When reproducing bugs or implementing fixes, use a single file rather than creating multiple files with different
versions
</FILE_SYSTEM_GUIDELINES>

<CODE_QUALITY>
* Write clean, efficient code with minimal comments. Avoid redundancy in comments: Do not repeat information that 
can be easily inferred from the code itself.
* When implementing solutions, focus on making the minimal changes needed to solve the problem.
* Before implementing any changes, first thoroughly understand the codebase through exploration.
* If you are adding a lot of code to a function or file, consider splitting the function or file into smaller 
pieces when appropriate.
* Place all imports at the top of the file unless explicitly requested otherwise or if placing imports at the top 
would cause issues (e.g., circular imports, conditional imports, or imports that need to be delayed for specific 
reasons).
</CODE_QUALITY>

<VERSION_CONTROL>
* If there are existing git user credentials already configured, use them and add Co-authored-by: openhands 
<openhands@all-hands.dev> to any commits messages you make. if a git config doesn't exist use "openhands" as the 
user.name and "openhands@all-hands.dev" as the user.email by default, unless explicitly instructed otherwise.
* Exercise caution with git operations. Do NOT make potentially dangerous changes (e.g., pushing to main, deleting 
repositories) unless explicitly asked to do so.
* When committing changes, use `git status` to see all modified files, and stage all files necessary for the 
commit. Use `git commit -a` whenever possible.
* Do NOT commit files that typically shouldn't go into version control (e.g.

✓ Conversation ready in: /Users/rajivshah/Code/openhands-deep-agent


## Step 7: Run a Research Query (uses Tavily search!)

In [7]:
# Ask the agent to research something - it will use Tavily search
conversation.send_message("""
Estimate the main profit drivers for NVIDIA in 2026.
Use the tavily_search tool to find current information, then summarize your findings.
""")
conversation.run()

Message from User ─────────────────────────────────────────────────────────────────────────────────────────────────


Estimate the main profit drivers for NVIDIA in 2026.
Use the tavily_search tool to find current information, then summarize your findings.


Tokens: ↑ input 0 • cache hit N/A • ↓ output 0 • $ 0.00

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search NVIDIA 2026 drivers: Blackwell/GB200/Rubin, guidance

Thought:
I'll search recent sources to identify NVIDIA's 2026 profit drivers.

Reasoning:
Action: SearchAction

Arguments:
  query: "NVIDIA 2026 profit drivers data center GPUs Blackwell GB200 Rubin 2026 timeline revenue guidance"
  kind: "SearchAction"

Tokens: ↑ input 680 • cache hit 0.00% •  reasoning 1.98K • ↓ output 2.48K • $ 0.0256

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search GB200/NVL72 shipments and 2026 impact

Action: SearchAction

Arguments:
  query: "NVIDIA Blackwell GB200 NVL72 shipments 2025 2026 revenue impact"
  kind: "SearchAction"

Tokens: ↑ input 680 • cache hit 0.00% •  reasoning 1.98K • ↓ output 2.48K • $ 0.0256

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search Spectrum-X/InfiniBand/NVLink 2026 revenue contribution

Action: SearchAction

Arguments:
  query: "NVIDIA networking Spectrum-X 2026 ramp InfiniBand NVLink revenue contribution"
  kind: "SearchAction"

Tokens: ↑ input 680 • cache hit 0.00% •  reasoning 1.98K • ↓ output 2.48K • $ 0.0256

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search Drive Thor production timing and revenue forecast 2026

Action: SearchAction

Arguments:
  query: "NVIDIA automotive Drive Thor production 2025 2026 revenue forecast"
  kind: "SearchAction"

Tokens: ↑ input 680 • cache hit 0.00% •  reasoning 1.98K • ↓ output 2.48K • $ 0.0256

...

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search AI Enterprise/NIM software revenue commentary 2025-2026

Reasoning:
Action: SearchAction

Arguments:
  query: "NVIDIA AI Enterprise revenue 2026 NIM software revenue commentary 2025 2026 earnings"
  kind: "SearchAction"

Tokens: ↑ input 24.85K • cache hit 75.21% •  reasoning 3.71K • ↓ output 4.33K • $ 0.0533

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: search
Result:
**Nvidia Earnings: Updates and Commentary November 2025**
Nvidia will release its fiscal third-quarter earnings report after the stock market closes on Wednesday, November 
19. Indeed, Nvidia CEO Jensen Huang said last month that the company has roughly $500 billion in total Blackwell 
and Rubin orders for this year and next – beating Wall Street's expectati
Source: https://www.kiplinger.com/investing/live/nvidia-earnings-live-updates-and-commentary-november-2025

**Nvidia's AI Reign Continues: Blockbuster Earnings Quell Bubble ...**
The semiconductor giant's stellar performance has not only sent its stock soaring but has also significantly 
rekindled faith in the broader artificial intelligence (AI) trade, effectively soothing widespread concerns about 
an impending "AI bubble." This report serves as a powerful testament to the e
Source: 
https://markets.financialcontent.com/wral/article/marketminute-2025-11-20-nvidias-ai-reign-continues-blockbuster-ea
rnings-quell-bubble-fears-fuel-market-rally

**Nvidia's record $57B revenue and upbeat forecast quiets AI bubble ...**
Nvidia founder and CEO Jensen Huang struck a bullish tone in the company’s third-quarter earnings. Revenue 
generated by Nvidia’s data center business was a record $51.2 billion, up 25% from the previous quarter and up 66% 
from a year ago. Nvidia’s CFO Colette Kress noted in a statement to shareholde
Source: https://techcrunch.com/2025/11/19/nvidias-record-57b-revenue-and-upbeat-forecast-quiets-ai-bubble-talk/

**[PDF] Investor Presentation Q2 FY26**
GAAP Non-GAAP Q2 FY26 Y/Y Q/Q Q2 FY26 Y/Y Q/Q Revenue $46,743 +56% +6% $46,743 +56% +6% Gross Margin (GM) 72.4% 
-2.7 pts +11.9 pts 72.7% -3.0 pts +11.7 pts GM excluding H20 related charges/ releases, net 72.3% +1.0 pt Operating
Income $28,440 +53% +31% $30,165 +51% +30% Net Income $26,422 +59% +41% 
Source: https://s201.q4cdn.com/141608511/files/doc_financials/2026/q2/NVDA-F2Q26-Quarterly-Presentation-FINAL.pdf

**Nvidia's $65 Billion Forecast Sends a Clear Message About the AI ...**
# Nvidia's $65 Billion Forecast Sends a Clear Message About the AI Boom. * Nvidia has been generating strong sales 
of its AI accelerators, and expects that to continue in its fiscal 2026 Q4 with estimated revenue of $65 billion. *
According to Nvidia's CEO, major platform shifts in technology are dr
Source: https://finance.yahoo.com/news/nvidias-65-billion-forecast-sends-142100577.html

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search Drive Thor 2026 production ramp and revenue contribution

Reasoning:
Action: SearchAction

Arguments:
  query: "NVIDIA Drive Thor 2026 production ramp automotive revenue FY2026 customers revenue contribution"
  kind: "SearchAction"

Tokens: ↑ input 38.8K • cache hit 82.15% •  reasoning 3.71K • ↓ output 4.38K • $ 0.0565

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: search
Result:
**NVIDIA Corp (NVDD.XD) Q2 FY2026 earnings call transcript**
Automotive revenue, which includes only in car compute revenue, was 586,000,000, up 69% year on year, primarily 
driven by self driving solutions
Source: https://finance.yahoo.com/quote/NVDD.XD/earnings/NVDD.XD-Q2-2026-earnings_call-351238.html/

**Nvidia - Company Analysis and Outlook Report (2026)**
Two customers accounted for 39% of Q2 FY2026 revenue, with four customers contributing 61% of total revenue. Major 
buyers (Microsoft, Amazon
Source: https://www.deepresearchglobal.com/p/nvidia-company-analysis-outlook-report

**NVIDIA 2026 Q3 Financial Results - by Ryan Smith - More Than Moore**
For the third quarter of NVIDIA’s 2026 fiscal year, the company booked a record $57.0b in revenue. With compute 
revenue growth having been largely stalled over the past couple of quarters, it would seem that NVIDIA has finally 
been able to clear up that jam – be it on the supply side or the pricing 
Source: https://morethanmoore.substack.com/p/nvidia-2026-q3-financial-results

**Nvidia's automotive revenue is quietly surging on explosive demand ...**
**The details:** The global tech company’s auto revenue over the past few quarters hasn’t just increased—it has 
literally catapulted, further highlighting why Nvidia has become such an integral part of the business. Nvidia’s 
big play in the auto sector is also being driven by the use of its AI techn
Source: 
https://news.dealershipguy.com/p/nvidia-s-automotive-revenue-is-quietly-surging-on-explosive-demand-for-ai-tech-202
5-06-02

**NVIDIA Q2 FY 2026 Results: Networking Steals the Spotlight - Futurum**
The News: NVIDIA (NASDAQ: NVDA) reported Q2 FY 2026 revenue of $46.7 billion, up 6% sequentially and 56% YoY, 
broadly in line with consensus
Source: https://futurumgroup.com/insights/nvidia-q2-fy-2026-earnings-networking-steals-the-spotlight/

Message from Agent ────────────────────────────────────────────────────────────────────────────────────────────────

I searched recent sources to identify NVIDIA’s 2026 profit drivers.

What the latest evidence says
- Data Center compute remains the core profit engine
  - Q3 FY2026 revenue hit $57B, with Data Center at $51.2B (+25% QoQ, +66% YoY), reflecting sustained demand for AI
training and inference capacity (TechCrunch; Q3 FY2026 coverage: 
https://techcrunch.com/2025/11/19/nvidias-record-57b-revenue-and-upbeat-forecast-quiets-ai-bubble-talk/).
  - NVIDIA guided to mid-70s non-GAAP gross margins in 2026 despite rising input costs, implying compute continues 
to dominate profit contribution (Motley Fool transcript summary: 
https://www.fool.com/earnings/call-transcripts/2025/11/19/nvidia-nvda-q3-2026-earnings-call-transcript/; Q2 FY26 
investor deck shows 72.7% non-GAAP GM: 
https://s201.q4cdn.com/141608511/files/doc_financials/2026/q2/NVDA-F2Q26-Quarterly-Presentation-FINAL.pdf).
  - Blackwell/GB200 ramp is the center of demand; multiple reports suggest orders extending well into mid-2026, 
though these are analyst/press claims rather than NVIDIA guidance (TheStreet commentary: 
https://www.thestreet.com/investing/stocks/nvidia-stocks-next-move-hinges-on-q4-earnings-blackwell-ramp; 
DeepResearchGlobal: https://www.deepresearchglobal.com/p/nvidia-company-analysis-outlook-report).
  - Grace CPU attach in GB200 systems is rising; third-party channel data points to a surge in Grace shipments, 
supporting performance and platform ASPs (CRN: 
https://www.crn.com/news/components-peripherals/2025/surge-in-nvidia-grace-cpus-contributes-to-record-arm-market-sh
are-estimate).

- Networking is the fastest-rising second profit pool
  - Networking revenue reached records in 2026, driven by NVLink, InfiniBand, and Spectrum‑X Ethernet.
    - Q2 FY2026 networking: $7.3B (Futurum Group: 
https://futurumgroup.com/insights/nvidia-q2-fy-2026-earnings-networking-steals-the-spotlight/).
    - Q3 FY2026 networking: $8.2B (+162% YoY) (SDXCentral: 
https://www.sdxcentral.com/news/nvidia-earnings-soar-as-chip-giants-networking-segment-continues-to-drive-ai-boom/;
also referenced in call transcripts).
  - NVIDIA highlights increasing Spectrum‑X adoption for Ethernet AI fabrics, expanding TAM beyond traditional 
InfiniBand (Yahoo Finance summary: https://finance.yahoo.com/news/nvidias-spectrum-x-adoption-rises-125600936.html;
AlphaSpread transcript summary: 
https://www.alphaspread.com/security/nasdaq/nvda/investor-relations/earnings-call/q2-2026).

- Software and platform revenues are small but margin-accretive and growing
  - NVIDIA is commercializing AI Enterprise, NIM microservices, and DGX Cloud with recurring revenue potential; 
management and analysts frame software as an emerging, higher‑margin mix component for 2026+ (TheStreet perspective
on recurring AI software: 
https://www.thestreet.com/investing/stocks/nvidia-stocks-next-move-hinges-on-q4-earnings-blackwell-ramp). While 
NVIDIA doesn’t break out software revenue, the mid-70s gross margins indicate software/platform mix is supportive 
of profit per dollar of revenue.

- Automotive ramps from a small base with DRIVE Thor
  - Automotive revenue in Q2 FY2026 was ~$586M, up ~69% YoY, reflecting ramping in-car compute (Yahoo Finance 
transcript summary: https://finance.yahoo.com/quote/NVDD.XD/earnings/NVDD.XD-Q2-2026-earnings_call-351238.html).
  - Thor targets production programs beginning 2025–2026 and beyond; technical targets indicate materially higher 
capability vs prior generations (TechInsights: https://www.techinsights.com/blog/nvidias-thor-hammers-atlan). This 
segment remains much smaller than Data Center but is a 2026+ growth vector with improving profitability as volumes 
scale.

- Gaming/ProViz stable, but not the primary profit driver
  - GeForce RTX 50-series continues shipping and getting ecosystem updates (NVIDIA CES 2026 page: 
https://www.nvidia.com/en-us/geforce/news/ce

## Why Persistence Matters

Your conversation is now saved to `.conversations/simple-agent-demo/`. If the notebook crashes or you close it:

```python
# Just re-create with same ID - automatically resumes!
conversation = Conversation(
    agent=agent,
    workspace=cwd,
    persistence_dir="./.conversations",
    conversation_id="simple-agent-demo",  # Same ID = auto-restore
)
conversation.send_message("What were we talking about?")
conversation.run()  # Continues from where it left off
```

### Compare to LangGraph

| OpenHands SDK | LangGraph |
|---------------|-----------|
| 2 parameters | Define StateGraph, nodes, edges, conditionals |
| No graph abstraction | Must think in graphs |
| Auto-saves every event | Need checkpointer class |
| ~5 lines | ~30-50 lines for same result |

```python
# LangGraph equivalent requires:
from langgraph.graph import StateGraph
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict): messages: list
builder = StateGraph(State)
builder.add_node("agent", call_model)
builder.add_node("tools", ToolNode(tools))
builder.add_conditional_edges("agent", should_continue, {...})
graph = builder.compile(checkpointer=SqliteSaver(...))
# ...and more
```

**OpenHands SDK: Persistence without the graph overhead.**